### Operations

- General
    - crop/merge (in one command)
    - warp
    - rasterize


- DEMs
    - color relief
    - hill shading
    - texture shading
    - various blends of color relief and shaded relief


- Landsat
    - stacking bands
    - autogain/white-balance
    - pansharpening


__Raw dataset types__
- A set of DEM tiles (NED13 tiles, elev48i100, etc)
- A set of Landsat scenes
- a shapefile


__Basal derived dataset types__
- a cropped and/or merged DEM
- a set of cropped and/or merged Landsat bands
- a rasterized shapefile


A basal derived dataset is necessarily the result of a single crop/merge of a raw dataset. The parameters of this single operation, combined with a raw dataset, define the derived dataset.

Not yet clear how to fit rasterizing shapefiles into this schema. 

### Landsat workflow
1. Crop and/or merge each band
2. Stack three of the merged bands
3. autogain/pansharpen/etc the resulting RGB image

In [ ]:
import os
import sys
import json
import glob
import rasterio
import numpy as np

sys.path.insert(0, '../')
from managers import managers

%load_ext autoreload
%autoreload 2

In [ ]:
L5_scene = '/home/keith/landsat-data/Landsat_1-5/LM01_L1TP_045034_19720916_20180429_01_T2/'
L8_scene = '/home/keith/landsat-data/LC08_L1TP_042034_20180907_20180912_01_T1//'

In [ ]:
s = managers.datasets.LandsatScene(L8_scene)

In [ ]:
rp = managers.DEMProject('../projects/test-dem', reset=True)

In [ ]:
rp.define_sources('../test/datasets/dem/n38w120/n38w120_100m.tif')

In [ ]:
rp.initialize(res=200)

In [ ]:
rp.hillshade()

In [ ]:
# test 'raw' landsat scenes
scenes = [
    'LC08_L1TP_041034_20180916_20180928_01_T1',
    'LC08_L1TP_042034_20180907_20180912_01_T1',
]

In [ ]:
gm = managers.RasterProject('../projects/test-landsat', reset=True)

In [ ]:
gm.define_sources(scenes, root='../test/datasets/landsat/')

In [ ]:
gm.sources[0].filepath(1)

In [ ]:
# bounds from merging-landsat.ipynb for central/southern sierra
bounds = managers.utils.shell(
    'echo "[-119.8, 36.2, -117.8, 38.2]" | %s transform - --dst-crs %s --precision 2' % \
     (gm.rio, gm.sources[0].filepath(1)))

bounds = json.loads(bounds)

In [ ]:
result = gm.initialize(400, bounds)

In [ ]:
gm.props